# An Analysis of Political Contributions During the 2020 House of Representatives Election

Goal of this notebook is to:
1. Webscrape the Open Secrets pages detailing the contribution sources for each candidate. 
2. Answer the following questions:
    * What does the overall distribution of funding sources look like?
    * Is there any detectable difference in contribution sources between Democrat and Republican candidates?
    * Do the funding sources for either the winning candidate or incumbent candidate differ from the other candidates?

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as BS
import time

In [2]:
## Prepare list of all URLs
# Initialize empty list for URLs
URLs = []

# Scrape table of state abbreviations and # of representatives per state from Britannica site
response = requests.get('https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120')
soup = BS(response.text)
states_reps = pd.read_html(str(soup.find('table')))[0].drop(50)

# Scrape abbreviation table from World Population Review site
response = requests.get('https://worldpopulationreview.com/states/state-abbreviations')
soup = BS(response.text)
states_abv = pd.read_html(str(soup.find('table')))[0]
states_abv = (states_abv.rename({'State':'state'},
                                axis='columns'))

# Merge the two read in dataframes
states = pd.merge(states_reps, states_abv, on='state').drop(columns=['Abbreviation','state'])

# Loop through states dataframe and create list of all URLs
for index in states.index:
    code = states.loc[index]['Code']
    districts = states.loc[index]['representatives']
    
    for dist in range(1, districts+1):
        URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id={}{}&spec=N'.format(code, str(dist).zfill(2))
        URLs.append(URL)

In [8]:
# Initialize empty dataframe to hold webscraped data
contributions = pd.DataFrame()

# Loop through lists of URLs and extract data from each
for URL in URLs:   
    response = requests.get(URL)
    soup = BS(response.text)
    strong = soup.findAll('strong')

    for i in range(len(strong)):
        # Format dataframe for current webpage
        cont_df = pd.read_html(str(soup.findAll('table')[1 + (3*i)]))[0].transpose()
        cont_df.columns = cont_df.iloc[0]
        cont_df = cont_df.drop(labels=['Type of Contribution', 'Percentage'], axis=0)
        cont_df = cont_df.reset_index(drop=True)
        
        # Split cand_info into separate elements
        cand_info = strong[i].text.split(' • ')
        
        # Assign candidate name to candidate column
        name = re.sub(r'\s\W\w\W', "", re.sub(r'\n{0,}\t{0,}', '', cand_info[0]))
        cont_df = cont_df.rename(index={0: name})
        
        # Create new column named party
        party = re.sub(r'\W', '', re.findall(r'\W\w\W', re.sub(r'\n', '', cand_info[0]))[0])
        cont_df.at[0, 'party'] =  party
        
        # Create column named incumbent and determine incumbent status
        for j in range(1, len(cand_info)):
            temp = re.sub(r'\s\W\w\W', "", re.sub(r'\n{0,}\t{0,}', '', cand_info[j]))

            if len(cand_info) == 1:
                cont_df.at[name, 'incumbent'] = False
                cont_df.at[0, 'winner'] = False

            if ('Incumbent' in temp):
                cont_df.at[0, 'incumbent'] = True
            elif ('Winner' in temp):
                cont_df.at[0, 'winner'] = True
                
    contributions = pd.concat([contributions, cont_df], ignore_index = True)

In [10]:
contributions.rename(index={0: name})

,Small Individual Contributions (≤ $200),Large Individual Contributions,PAC Contributions*,Candidate self-financing,Other,party,incumbent,winner
Lynnette Grey Bull,"$12,895","$37,955",$0,"$29,245",$0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN
2,"$15,307","$27,105","$2,032","$10,575","$1,030",NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN
4,"$36,483","$10,430",$108,$700,"$2,552",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN
866,"$169,380","$221,453","$30,183",$0,"-$4,038",NaN,NaN,NaN
867,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN
868,"$64,222","$65,975","$2,800",$0,"$1,600",NaN,NaN,NaN
